In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

In [3]:
# Load YOLO model
model = YOLO("yolov8n.pt")  

 

# Load video
video_path = "busdetection.mp4"
cap = cv2.VideoCapture(video_path)

# Get video dimensions
frame_width = int(cap.get(3))  
frame_height = int(cap.get(4))  

# Detect non-black area (road) dynamically
def get_road_region(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)  # Detect non-black pixels
    x, y, w, h = cv2.boundingRect(thresh)  # Get bounding box of road area
    return (x, y, x + w, y + h)  # (x1, y1, x2, y2)

# Process first frame to find road region
ret, frame = cap.read()
road_x1, road_y1, road_x2, road_y2 = get_road_region(frame)

# Define bus lane inside detected road area
bus_lane_x1 = int(road_x1 + 0.10 * (road_x2 - road_x1))  
bus_lane_x2 = int(road_x1 + 0.90 * (road_x2 - road_x1))  
bus_lane_y1 = road_y1  
bus_lane_y2 = road_y2  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    # Draw detected road boundary
    cv2.rectangle(frame, (road_x1, road_y1), (road_x2, road_y2), (255, 255, 255), 2)

    # Draw bus lane inside detected road
    cv2.rectangle(frame, (bus_lane_x1, bus_lane_y1), (bus_lane_x2, bus_lane_y2), (0, 255, 0), 2)
    cv2.putText(frame, "Bus Lane", (bus_lane_x1, bus_lane_y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # for r in results:
    #     for box in r.boxes:
    #         cls = int(box.cls[0])  
    #         label = r.names[cls]  

    #         x1, y1, x2, y2 = map(int, box.xyxy[0])
    #         center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

    #         # Check if the detected vehicle is in the bus lane
    #         if bus_lane_x1 < center_x < bus_lane_x2 and bus_lane_y1 < center_y < bus_lane_y2:
    #             if label != "bus" or label != "truck" :  # If it's not a bus
    #                 cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  
    #                 cv2.putText(frame, f"Unauthorized: {label}", (x1, y1 - 10),
    #                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    for r in results:
         for box in r.boxes:
            cls = int(box.cls[0])
            label = r.names[cls]

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            width = x2 - x1
            height = y2 - y1

            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

            # Check if the vehicle is inside the bus lane
            if bus_lane_x1 < center_x < bus_lane_x2 and bus_lane_y1 < center_y < bus_lane_y2:
                if label in ["truck", "car","person","bike","van" ,"auto" ]:  # Authorized vehicles
                    color = (0, 255, 0)  # Green
                    text = "Authorized"
                else:  # Unauthorized vehicles
                    color = (0, 0, 255)  # Red
                    text = f"Unauthorized: Bus"

                # Draw the bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, text, (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)


    cv2.imshow("Bus Monitoring", frame)
 
    if cv2.waitKey(1) & 0xFF == ord("q"):  
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 2 cars, 1 bus, 2 trucks, 106.7ms
Speed: 3.9ms preprocess, 106.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 bus, 1 truck, 99.9ms
Speed: 2.3ms preprocess, 99.9ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 2 buss, 97.8ms
Speed: 2.4ms preprocess, 97.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 2 trucks, 88.6ms
Speed: 2.8ms preprocess, 88.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 2 buss, 85.1ms
Speed: 2.4ms preprocess, 85.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 2 buss, 1 truck, 85.8ms
Speed: 2.4ms preprocess, 85.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 2 buss, 2 trucks, 84.2ms
Speed: 2.7ms preprocess, 84.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 2 buss